In [1]:
# Import packages
import numpy as np
import pickle as pk
import qiskit as qk
from qiskit import IBMQ, Aer
import qiskit.providers.aer.noise as noise
import random
import warnings
warnings.filterwarnings('ignore')

# Load IBM backend and provider
backendQASM = Aer.get_backend('qasm_simulator')
IBMQ.load_account()
provider = IBMQ.get_provider(hub = 'ibm-q')

# Create thermal noise model
noise_model = noise.NoiseModel()
t1 = 30000
t2 = 30000
time_u3 = 500
errors_u3 = noise.thermal_relaxation_error(t1, t2, time_u3)
noise_model.add_quantum_error(errors_u3, 'u3', [0])

qr = qk.QuantumRegister(1)
cr = qk.ClassicalRegister(1)

def u3Circ(theta, phi, lamda):
    circ = qk.QuantumCircuit(qr,cr)
    for k in range(len(theta)):
        circ.u3(theta[k], phi[k], lamda[k], 0)
    circ.measure(0,0)
    return circ

def getProb(state, counts, shots):
    if state in counts.keys(): return counts[state]/shots
    else: return 0

qk.__version__

'0.15.1'

In [3]:
# Create gate parameter list
num_gates, data_size, shots = 100, 100, 8192
X = []
for k in range(1, num_gates+1):
    for n in range(data_size):
        temp = []
        for _ in range(k):
            temp += [round(np.random.rand()*2*np.pi-np.pi, 5), 
                     round(np.random.rand()*2*np.pi-np.pi, 5),
                     round(np.random.rand()*2*np.pi-np.pi, 5)]
        X.append(temp)
random.shuffle(X)

# Create circuit list
circ_list = []
for x in X:
    circ_list.append(u3Circ(x[::3], x[1::3], x[2::3]))
len(circ_list)

10000

In [4]:
# Run circuits and return results
results = qk.execute(circ_list, backend=backendQASM, noise_model=noise_model, shots=shots, optimization_level=0).result()
Y = [getProb('0', results.get_counts(k), shots) for k in range(num_gates*data_size)]

# Buffer input data
for k in range(len(X)):
    X[k] = X[k] + [0.0]*(300-len(X[k]))

In [5]:
# Save data
with open('dataset_qasm.p', 'wb') as f:
    pk.dump({'X':X,'Y':Y}, f)